In [1]:
import torch
import pandas as pd
import time
from torch.utils.data import Dataset, TensorDataset
from torch.nn.utils.rnn import pad_sequence

import torch.nn as nn

from transformers import LongformerForSequenceClassification, LongformerTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset
# 解决服务器挂掉的问题
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
import transformers
print(transformers.__version__)

4.39.3


In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [4]:
MAX_LEN = 1024
MAX_EPOCHS = 100
BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_CLASSES = 2
WEIGTH_DECAY = 1e-3

In [5]:
model_path = "D:/LTX/code_comment_inconsistency_detection/longformer-base-4096"
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_CLASSES)
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels=NUM_CLASSES)

tokenizer = LongformerTokenizer.from_pretrained(model_path, do_lower_case=False)

print(sum([i.nelement() for i in model.parameters()]) / 10000)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at D:/LTX/code_comment_inconsistency_detection/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


14866.0994


处理数据  提取出 old_code_raw old_comment_raw label 三列，并且对old_code_raw的值中的\n去掉

In [6]:
# 全用 和 只用summary比较
# 只用summary的效果可以拿来做对比了
def retrieve_train_data():
    train_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/train.json")
    train_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/train.json")
    train_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/train.json")
    train_df = pd.concat([train_summary,train_param, train_return], axis=0)
    # train_df = train_df[:8397]
    return train_df
def retrieve_valid_data():
    valid_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/valid.json")
    valid_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/valid.json")
    valid_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/valid.json")
    valid_df = pd.concat([valid_summary,valid_param, valid_return ], axis=0)
    # valid_df = valid_df[:800]
    return valid_df
def retrieve_test_data():
    test_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    # test_df = test_df[:1000]
    return test_df

In [7]:
train_df = retrieve_train_data()
valid_df = retrieve_valid_data()
test_df = retrieve_test_data()
train_df.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,grails-plugins_grails-plugin-converters-5-Asso...,1,Summary,Parses the given JSON and returns ether a JSON...,"[parses, the, given, json, and, returns, ether...",Parses the given JSON and returns either a JSO...,"[parses, the, given, json, and, returns, eithe...","[<REPLACE_OLD>, ether, <REPLACE_NEW>, either, ...",public static JSONElement parse(InputStrea...,"[public, static, jsonelement, parse, (, input,...",public static JSONElement parse(InputStrea...,"[public, static, jsonelement, parse, (, input,...","[<KEEP>, public, static, jsonelement, parse, (...","[<KEEP>, public, <KEEP>, static, <KEEP>, jsone..."
1,jitsi_jitsi-4343-FirstSentence-0,0,Summary,Loads an image from a given image identifier.,"[loads, an, image, from, a, given, image, iden...",Loads an image from a given image identifier.,"[loads, an, image, from, a, given, image, iden...",[],public static byte[] getImageInBytes(Strin...,"[public, static, byte, [, ], get, image, in, b...",public static byte[] getImageInBytes(Strin...,"[public, static, byte, [, ], get, image, in, b...","[<KEEP>, public, static, byte, [, ], get, imag...","[<KEEP>, public, <KEEP>, static, <KEEP>, byte,..."
2,dropwizard_metrics-26-Associations-FirstSentence,1,Summary,Creates a new CounterMetric and registers it ...,"[creates, a, new, counter, metric, and, regist...",Creates a new com.yammer.metrics.core.Counter...,"[creates, a, new, com, ., yammer, ., metrics, ...","[<INSERT_OLD_KEEP_BEFORE>, new, <INSERT_NEW_KE...",public static CounterMetric newCounter(Cla...,"[public, static, counter, metric, new, counter...",public static Counter newCounter(Class<?> ...,"[public, static, counter, new, counter, (, cla...","[<KEEP>, public, static, counter, <KEEP_END>, ...","[<KEEP>, public, <KEEP>, static, <KEEP>, count..."
3,google_ExoPlayer-92-FirstSentence-0,0,Summary,Derives a sample format corresponding to a giv...,"[derives, a, sample, format, corresponding, to...",Derives a sample format corresponding to a giv...,"[derives, a, sample, format, corresponding, to...",[],private static Format getSampleFormat(Format...,"[private, static, format, get, sample, format,...",private static Format getSampleFormat(Format...,"[private, static, format, get, sample, format,...","[<KEEP>, private, static, format, get, sample,...","[<KEEP>, private, <KEEP>, static, <KEEP>, form..."
4,slachiewicz_orekit-main-661-Associations-First...,1,Summary,Revert a rotation/rotation rate pair.,"[revert, a, rotation, /, rotation, rate, pair, .]",Revert a rotation/rotation rate/ rotation acce...,"[revert, a, rotation, /, rotation, rate, /, ro...","[<REPLACE_OLD>, pair, <REPLACE_NEW>, /, rotati...",public AngularCoordinates revert() {\n ...,"[public, angular, coordinates, revert, (, ), {...",public AngularCoordinates revert() {\n ...,"[public, angular, coordinates, revert, (, ), {...","[<KEEP>, public, angular, coordinates, revert,...","[<KEEP>, public, <KEEP>, angular, <KEEP>, coor..."


In [8]:
valid_df.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,todoroo_astrid-987-FirstSentence-0,1,Summary,Return SQL selector query for getting tasks wi...,"[return, sql, selector, query, for, getting, t...",Return SQL selector query for getting tasks wi...,"[return, sql, selector, query, for, getting, t...","[<INSERT_OLD_KEEP_BEFORE>, tag, <INSERT_NEW_KE...",public QueryTemplate queryTemplate(Cri...,"[public, query, template, query, template, (, ...",public static QueryTemplate queryTempl...,"[public, static, query, template, query, templ...","[<KEEP>, public, <KEEP_END>, <INSERT>, static,...","[<KEEP>, public, <INSERT>, static, <KEEP>, que..."
1,Red5_red5_server-43-FirstSentence-0,0,Summary,Return period of ghost connections cleanup tas...,"[return, period, of, ghost, connections, clean...",Return period of ghost connections cleanup tas...,"[return, period, of, ghost, connections, clean...",[],public int getGhostConnsCleanupPeriod() {\...,"[public, int, get, ghost, conns, cleanup, peri...",public int getGhostConnsCleanupPeriod() {\...,"[public, int, get, ghost, conns, cleanup, peri...","[<KEEP>, public, int, get, ghost, conns, clean...","[<KEEP>, public, <KEEP>, int, <KEEP>, get, <KE..."
2,nickman_Rindle-11-Associations-FirstSentence,1,Summary,Allocates an initialized and initially unlocke...,"[allocates, an, initialized, and, initially, u...",Allocates an initialized and initially unlocke...,"[allocates, an, initialized, and, initially, u...","[<INSERT_OLD_KEEP_BEFORE>, unlocked, <INSERT_N...",\tpublic static long allocateSpinLock() {\r\n\...,"[public, static, long, allocate, spin, lock, (...",\tpublic static SpinLock allocateSpinLock() {\...,"[public, static, spin, lock, allocate, spin, l...","[<KEEP>, public, static, <KEEP_END>, <REPLACE_...","[<KEEP>, public, <KEEP>, static, <REPLACE_OLD>..."
3,h2oai_h2o_2-427-FirstSentence-0,0,Summary,Rebalance a frame for load balancing,"[rebalance, a, frame, for, load, balancing]",Rebalance a frame for load balancing,"[rebalance, a, frame, for, load, balancing]",[],"private Frame reBalance(final Frame fr, bool...","[private, frame, re, balance, (, final, frame,...",private static Frame reBalance(final Frame f...,"[private, static, frame, re, balance, (, final...","[<KEEP>, private, <KEEP_END>, <INSERT>, static...","[<KEEP>, private, <INSERT>, static, <KEEP>, fr..."
4,sonatype_sonatype-aether-11-Associations-First...,1,Summary,Sets the host of this proxy.,"[sets, the, host, of, this, proxy, .]",Sets the host of the proxy.,"[sets, the, host, of, the, proxy, .]","[<REPLACE_OLD>, this, <REPLACE_NEW>, the, <REP...",public Proxy setHost( String host )\n {...,"[public, proxy, set, host, (, string, host, ),...",public Proxy setHost( String host )\n {...,"[public, proxy, set, host, (, string, host, ),...","[<KEEP>, public, proxy, set, host, (, string, ...","[<KEEP>, public, <KEEP>, proxy, <KEEP>, set, <..."


In [9]:
def format_data(df):
    old_code_raw = df['new_code_raw']
    old_code_raw = old_code_raw.values
    old_code_raw = [str(ele) for ele in old_code_raw]
       
    multi_line_old_code = []
    for i in range(len(old_code_raw)):
        multi_line_test = old_code_raw[i].replace('\n', ' ')   # 去掉\n
        multi_line_test = ' '.join(multi_line_test.split())    # 把多余空格变成一个空格
        multi_line_old_code.append(multi_line_test) 
     
    old_comment_raw = df['old_comment_raw']
    old_comment_raw = old_comment_raw.values
    old_comment_raw = [str(ele) for ele in old_comment_raw]
    multi_line_old_comment = []
    for i in range(len(old_comment_raw)):
        multi_line_test = ' '.join(old_comment_raw[i].split())    # 把多余空格变成一个空格
        multi_line_old_comment.append(multi_line_test)  
    
    df['new_code_raw'] = multi_line_old_code
    df['old_comment_raw'] = multi_line_old_comment
    
    return df

In [10]:
train_df_clean = format_data(train_df)
train_df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,grails-plugins_grails-plugin-converters-5-Asso...,1,Summary,Parses the given JSON and returns ether a JSON...,"[parses, the, given, json, and, returns, ether...",Parses the given JSON and returns either a JSO...,"[parses, the, given, json, and, returns, eithe...","[<REPLACE_OLD>, ether, <REPLACE_NEW>, either, ...",public static JSONElement parse(InputStrea...,"[public, static, jsonelement, parse, (, input,...",public static JSONElement parse(InputStream is...,"[public, static, jsonelement, parse, (, input,...","[<KEEP>, public, static, jsonelement, parse, (...","[<KEEP>, public, <KEEP>, static, <KEEP>, jsone..."
1,jitsi_jitsi-4343-FirstSentence-0,0,Summary,Loads an image from a given image identifier.,"[loads, an, image, from, a, given, image, iden...",Loads an image from a given image identifier.,"[loads, an, image, from, a, given, image, iden...",[],public static byte[] getImageInBytes(Strin...,"[public, static, byte, [, ], get, image, in, b...",public static byte[] getImageInBytes(String im...,"[public, static, byte, [, ], get, image, in, b...","[<KEEP>, public, static, byte, [, ], get, imag...","[<KEEP>, public, <KEEP>, static, <KEEP>, byte,..."
2,dropwizard_metrics-26-Associations-FirstSentence,1,Summary,Creates a new CounterMetric and registers it u...,"[creates, a, new, counter, metric, and, regist...",Creates a new com.yammer.metrics.core.Counter...,"[creates, a, new, com, ., yammer, ., metrics, ...","[<INSERT_OLD_KEEP_BEFORE>, new, <INSERT_NEW_KE...",public static CounterMetric newCounter(Cla...,"[public, static, counter, metric, new, counter...",public static Counter newCounter(Class<?> klas...,"[public, static, counter, new, counter, (, cla...","[<KEEP>, public, static, counter, <KEEP_END>, ...","[<KEEP>, public, <KEEP>, static, <KEEP>, count..."
3,google_ExoPlayer-92-FirstSentence-0,0,Summary,Derives a sample format corresponding to a giv...,"[derives, a, sample, format, corresponding, to...",Derives a sample format corresponding to a giv...,"[derives, a, sample, format, corresponding, to...",[],private static Format getSampleFormat(Format...,"[private, static, format, get, sample, format,...",private static Format getSampleFormat(Format c...,"[private, static, format, get, sample, format,...","[<KEEP>, private, static, format, get, sample,...","[<KEEP>, private, <KEEP>, static, <KEEP>, form..."
4,slachiewicz_orekit-main-661-Associations-First...,1,Summary,Revert a rotation/rotation rate pair.,"[revert, a, rotation, /, rotation, rate, pair, .]",Revert a rotation/rotation rate/ rotation acce...,"[revert, a, rotation, /, rotation, rate, /, ro...","[<REPLACE_OLD>, pair, <REPLACE_NEW>, /, rotati...",public AngularCoordinates revert() {\n ...,"[public, angular, coordinates, revert, (, ), {...",public AngularCoordinates revert() { return ne...,"[public, angular, coordinates, revert, (, ), {...","[<KEEP>, public, angular, coordinates, revert,...","[<KEEP>, public, <KEEP>, angular, <KEEP>, coor..."


In [11]:
valid_df_clean = format_data(valid_df)
valid_df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,todoroo_astrid-987-FirstSentence-0,1,Summary,Return SQL selector query for getting tasks wi...,"[return, sql, selector, query, for, getting, t...",Return SQL selector query for getting tasks wi...,"[return, sql, selector, query, for, getting, t...","[<INSERT_OLD_KEEP_BEFORE>, tag, <INSERT_NEW_KE...",public QueryTemplate queryTemplate(Cri...,"[public, query, template, query, template, (, ...",public static QueryTemplate queryTemplate(Crit...,"[public, static, query, template, query, templ...","[<KEEP>, public, <KEEP_END>, <INSERT>, static,...","[<KEEP>, public, <INSERT>, static, <KEEP>, que..."
1,Red5_red5_server-43-FirstSentence-0,0,Summary,Return period of ghost connections cleanup tas...,"[return, period, of, ghost, connections, clean...",Return period of ghost connections cleanup tas...,"[return, period, of, ghost, connections, clean...",[],public int getGhostConnsCleanupPeriod() {\...,"[public, int, get, ghost, conns, cleanup, peri...",public int getGhostConnsCleanupPeriod() { retu...,"[public, int, get, ghost, conns, cleanup, peri...","[<KEEP>, public, int, get, ghost, conns, clean...","[<KEEP>, public, <KEEP>, int, <KEEP>, get, <KE..."
2,nickman_Rindle-11-Associations-FirstSentence,1,Summary,Allocates an initialized and initially unlocke...,"[allocates, an, initialized, and, initially, u...",Allocates an initialized and initially unlocke...,"[allocates, an, initialized, and, initially, u...","[<INSERT_OLD_KEEP_BEFORE>, unlocked, <INSERT_N...",\tpublic static long allocateSpinLock() {\r\n\...,"[public, static, long, allocate, spin, lock, (...",public static SpinLock allocateSpinLock() { lo...,"[public, static, spin, lock, allocate, spin, l...","[<KEEP>, public, static, <KEEP_END>, <REPLACE_...","[<KEEP>, public, <KEEP>, static, <REPLACE_OLD>..."
3,h2oai_h2o_2-427-FirstSentence-0,0,Summary,Rebalance a frame for load balancing,"[rebalance, a, frame, for, load, balancing]",Rebalance a frame for load balancing,"[rebalance, a, frame, for, load, balancing]",[],"private Frame reBalance(final Frame fr, bool...","[private, frame, re, balance, (, final, frame,...","private static Frame reBalance(final Frame fr,...","[private, static, frame, re, balance, (, final...","[<KEEP>, private, <KEEP_END>, <INSERT>, static...","[<KEEP>, private, <INSERT>, static, <KEEP>, fr..."
4,sonatype_sonatype-aether-11-Associations-First...,1,Summary,Sets the host of this proxy.,"[sets, the, host, of, this, proxy, .]",Sets the host of the proxy.,"[sets, the, host, of, the, proxy, .]","[<REPLACE_OLD>, this, <REPLACE_NEW>, the, <REP...",public Proxy setHost( String host )\n {...,"[public, proxy, set, host, (, string, host, ),...",public Proxy setHost( String host ) { return n...,"[public, proxy, set, host, (, string, host, ),...","[<KEEP>, public, proxy, set, host, (, string, ...","[<KEEP>, public, <KEEP>, proxy, <KEEP>, set, <..."


In [12]:
test_df_clean = format_data(test_df)
test_df_clean.head()

,id,label,comment_type,old_comment_raw,old_comment_subtokens,new_comment_raw,new_comment_subtokens,span_minimal_diff_comment_subtokens,old_code_raw,old_code_subtokens,new_code_raw,new_code_subtokens,span_diff_code_subtokens,token_diff_code_subtokens
0,apache_calcite-896-FirstSentence-0,1,Summary,Creates elastic node as single member of a clu...,"[creates, elastic, node, as, single, member, o...",Creates an instance with existing settings,"[creates, an, instance, with, existing, settings]","[<REPLACE_OLD>, elastic, node, as, single, mem...",public static EmbeddedElasticsearchNode crea...,"[public, static, embedded, elasticsearch, node...",private static EmbeddedElasticsearchNode creat...,"[private, static, embedded, elasticsearch, nod...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private...","[<REPLACE_OLD>, public, <REPLACE_NEW>, private..."
1,hibernate_hibernate_orm-1601-FirstSentence-0,0,Summary,Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",Check whether any insertion or deletion action...,"[check, whether, any, insertion, or, deletion,...",[],\tpublic boolean areInsertionsOrDeletionsQueue...,"[public, boolean, are, insertions, or, deletio...",public boolean areInsertionsOrDeletionsQueued(...,"[public, boolean, are, insertions, or, deletio...","[<KEEP>, public, boolean, are, insertions, or,...","[<KEEP>, public, <KEEP>, boolean, <KEEP>, are,..."
2,apache_giraph-33-Associations-FirstSentence,1,Summary,Marshal the aggregator values of to a JSONArra...,"[marshal, the, aggregator, values, of, to, a, ...",Marshal the aggregator values of the worker to...,"[marshal, the, aggregator, values, of, the, wo...","[<INSERT_OLD_KEEP_BEFORE>, of, <INSERT_NEW_KEE...",private JSONArray marshalAggregatorValues(lo...,"[private, jsonarray, marshal, aggregator, valu...",private byte[] marshalAggregatorValues(long su...,"[private, byte, [, ], marshal, aggregator, val...","[<KEEP>, private, <KEEP_END>, <REPLACE_OLD>, j...","[<KEEP>, private, <REPLACE_OLD>, jsonarray, <R..."
3,apache_calcite-677-FirstSentence-0,0,Summary,Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",Returns a list of the types of the fields in a...,"[returns, a, list, of, the, types, of, the, fi...",[],public static List<RelDataType> getFieldType...,"[public, static, list, <, rel, data, type, >, ...",public static List<RelDataType> getFieldTypeLi...,"[public, static, list, <, rel, data, type, >, ...","[<KEEP>, public, static, list, <, rel, data, t...","[<KEEP>, public, <KEEP>, static, <KEEP>, list,..."
4,apache_calcite-315-Associations-FirstSentence,1,Summary,Create an instance based on current maven prof...,"[create, an, instance, based, on, current, mav...",Creates an instance based on current maven pro...,"[creates, an, instance, based, on, current, ma...","[<REPLACE_OLD>, create, <REPLACE_NEW>, creates...",static MongoDatabaseRule create() {\n fin...,"[static, mongo, database, rule, create, (, ), ...",static MongoDatabasePolicy create() { final Mo...,"[static, mongo, database, policy, create, (, )...","[<KEEP>, static, mongo, database, <KEEP_END>, ...","[<KEEP>, static, <KEEP>, mongo, <KEEP>, databa..."


In [13]:
class CocoDataset(Dataset):
    def __init__(self, df):
        self.df = df
        model_path = "D:/LTX/code_comment_inconsistency_detection/longformer-base-4096"
        self.tokenizer = LongformerTokenizer.from_pretrained(model_path, do_lower_case=False)        
        self.data = self.load_data(self.df)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def load_data(self, df):
        token_ids = []
        mask_ids = []
        seg_ids = []
        labels = []
        
        code_list = df['new_code_raw'].to_list() 
        comment_list = df['old_comment_raw'].to_list()
        label_list = df['label'].to_list()

        for (code, comment, label) in zip(code_list, comment_list, label_list):
            code_id = self.tokenizer.encode(code, add_special_tokens=False, truncation=True, max_length=256)
            comment_id = self.tokenizer.encode(comment, add_special_tokens=False, truncation=True, max_length=256)

            # want [CLS] comment tokens [SEP] code tokens [SEP]
            pair_token_ids = [self.tokenizer.cls_token_id] + comment_id + [self.tokenizer.sep_token_id] + code_id + [self.tokenizer.sep_token_id]
            pair_token_ids = self.truncate(pair_token_ids)
            code_len = len(code_id)
            comment_len = len(comment_id)
            
            attention_mask_ids = torch.tensor([1] * (code_len + comment_len + 3)) # mask padded values
            segment_ids = torch.tensor([0] * (comment_len + 2) + [1] * (code_len + 1)) # sentence 0 (comment) and sentence 1 (code)
            
            attention_mask_ids = self.truncate(attention_mask_ids)
            segment_ids = self.truncate(segment_ids)
            
            token_ids.append(torch.tensor(pair_token_ids))
            mask_ids.append(attention_mask_ids)
            seg_ids.append(segment_ids)
            labels.append(label)
            
        token_ids = pad_sequence(token_ids, batch_first=True)
        mask_ids = pad_sequence(mask_ids, batch_first=True)
        seg_ids = pad_sequence(seg_ids, batch_first=True)
        labels = torch.tensor(labels)
        
        dataset = TensorDataset(token_ids, mask_ids, seg_ids, labels)
        return dataset

    def truncate(self, ids):
        return ids[:MAX_LEN] if len(ids) > 512 else ids

In [14]:
train_data = CocoDataset(train_df_clean)
valid_data = CocoDataset(valid_df_clean)
test_data = CocoDataset(test_df_clean)

train_data,valid_data,test_data

(<__main__.CocoDataset at 0x268fe54a830>,
 <__main__.CocoDataset at 0x268c48501f0>)

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(predicted_labels, gold_labels):
    predicted_labels = [label.item() for label in predicted_labels]
    gold_labels = [label.item() for label in gold_labels]

    assert len(predicted_labels) == len(gold_labels)

    precision = precision_score(gold_labels, predicted_labels, zero_division=0)
    recall = recall_score(gold_labels, predicted_labels, zero_division=0)
    f1 = f1_score(gold_labels, predicted_labels, zero_division=0)
    accuracy = accuracy_score(gold_labels, predicted_labels) 

    return precision, recall, f1, accuracy

In [16]:
def test(model, test_data_t):
    
    test_data = CocoDataset(test_data_t)
    test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

    model.eval()
    test_loss = 0.0
    predictions = []
    gold_labels = []

    with torch.no_grad():
        for batch_idx, (sequence, attention_masks, token_type_ids, labels) in enumerate(test_loader):
            sequence = sequence.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)

            outputs = model(sequence, attention_mask=attention_masks, token_type_ids=token_type_ids, labels=labels)
            loss = outputs.loss
            prediction = outputs.logits

            test_loss += loss.item()
            prediction = torch.argmax(prediction, dim=-1)

            predictions.extend(prediction)
            gold_labels.extend(labels)
        
    test_loss = test_loss / len(test_loader)
    test_precision, test_recall, test_f1, test_acc = compute_metrics(predictions, gold_labels)

    print(f"test_loss: {test_loss:.3f} test_precision: {test_precision:.3f} test_recall: {test_recall:.3f} test_f1: {test_f1:.3f} test_acc: {test_acc:.3f}")

In [17]:
from transformers.utils.notebook import format_time
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
def train(model,train_data_,valid_data_,patience=10):


    train_loader = DataLoader(dataset=train_data_, batch_size=BATCH_SIZE)
    valid_loader = DataLoader(dataset=valid_data_, batch_size=BATCH_SIZE)

    best_f1 = 0.0
    epochs_without_improvement = 0
    
    total_steps = len(train_loader) * MAX_EPOCHS
    print("   Train batch size = {}".format(BATCH_SIZE))
    print("   Total steps = {}".format(total_steps))
    print(f"   Training Start!")
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    train_losses = []
    valid_losses = []
    times = 0
    for epoch in range(MAX_EPOCHS):
        # 每个周期输出一个txt，检查valid的预测数据
        times = times+1
        start = time.time()
        model.to(device)
        model.train()
    
        # string_colmn_name = "input_data    label    prediction_label"
        # file_name = 'check_newdata_bert_valid_dataset_epoch_{}'
        # with open(file_name.format(times)+'.txt', 'w') as f:
        #     f.write(string_colmn_name)
            
        train_loss = 0.0
        predictions = []
        gold_labels = []
        for batch_idx, (sequence, attention_masks, token_type_ids, labels) in enumerate(tqdm(train_loader, desc="Training", leave=True)):
            sequence = sequence.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            outputs = model(sequence, attention_mask=attention_masks,labels=labels)
            loss = outputs.loss
            prediction = outputs.logits

            train_loss += loss.item()
            prediction = torch.argmax(prediction, dim=-1)

            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)  # 梯度截断

            optimizer.step()
            optimizer.zero_grad()
#             scheduler.step()

            predictions.extend(prediction)
            gold_labels.extend(labels)

        train_loss = train_loss / len(train_loader)
        train_precision, train_recall, train_f1, train_acc = compute_metrics(predictions, gold_labels)
        train_time = format_time(time.time()-start)

        model.eval()
        valid_loss = 0.0
        predictions = []
        gold_labels = []

        with torch.no_grad():
            for batch_idx, (sequence, attention_masks, token_type_ids, labels) in enumerate(valid_loader):
                sequence = sequence.to(device)
                attention_masks = attention_masks.to(device)
                token_type_ids = token_type_ids.to(device)
                labels = labels.to(device)
                
                outputs = model(sequence, attention_mask=attention_masks,  labels=labels)
                loss = outputs.loss
                prediction = outputs.logits
                
                valid_loss += loss.item()
                prediction = torch.argmax(prediction, dim=1)

                predictions.extend(prediction)
                gold_labels.extend(labels)

                labels_cpu = labels.cpu()
                prediction_cpu = prediction.cpu()
                input_data = str([tokenizer.decode(ids,skip_special_tokens=True) for ids in sequence])
                # with open(file_name.format(times) +'.txt', 'a') as f:
                #     f.write(input_data)
                #     f.write('%s\n' % labels_cpu.numpy().tolist())
                #     f.write('%s\n' % prediction_cpu.numpy().tolist())
     
        valid_time = format_time(time.time() - start)
        valid_loss = valid_loss / len(valid_loader)
        valid_precision, valid_recall, valid_f1, valid_acc = compute_metrics(predictions, gold_labels)
        
        if valid_f1 != best_f1:
            if valid_f1 > best_f1:
                best_f1 = valid_f1
                print(f"New best validation f1 of {valid_f1:.3f}. Saving model.")
                torch.save(model, "save_longformerModel.pt")
            epochs_without_improvement = 0      
        else:
            epochs_without_improvement += 1
            
        # 如果验证集上的f1连续patience个epoch没有变化，则停止训练
        if epochs_without_improvement == patience:
            print('Early stopping at epoch {}...'.format(epoch+1))
            break

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        end = time.time()
        hours, rem = divmod(end - start, 3600)
        min, sec = divmod(rem, 60)

        print(f"Epoch {epoch + 1}: train_loss: {train_loss:.3f} train_precision: {train_precision:.3f} train_recall: {train_recall:.3f} train_f1: {train_f1:.3f} train_acc: {train_acc:.3f}")
        print(f"\t valid_loss: {valid_loss:.3f} valid_precision: {valid_precision:.3f} valid_recall: {valid_recall:.3f} valid_f1: {valid_f1:.3f} valid_acc: {valid_acc:.3f}")
        print("\t {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(min), sec))

    print('   Training Completed!')
    return train_losses,valid_losses

In [ ]:
train_losses,valid_losses = train(model,train_data,valid_data)

   Train batch size = 8
   Total steps = 412400
   Training Start!


Training:   0%|                                                                               | 0/4124 [00:00<?, ?it/s]Initializing global attention on CLS token...
Input ids are automatically padded from 515 to 1024 to be a multiple of `config.attention_window`: 512
Training: 100%|██████████████████████████████████████████████████████████████████| 4124/4124 [1:17:38<00:00,  1.13s/it]
Input ids are automatically padded from 398 to 512 to be a multiple of `config.attention_window`: 512


New best validation f1 of 0.522. Saving model.
Epoch 1: train_loss: 0.569 train_precision: 0.731 train_recall: 0.640 train_f1: 0.683 train_acc: 0.702
	 valid_loss: 0.703 valid_precision: 0.758 valid_recall: 0.398 valid_f1: 0.522 valid_acc: 0.636
	 01:19:50.66


Training: 100%|██████████████████████████████████████████████████████████████████| 4124/4124 [1:17:56<00:00,  1.13s/it]


New best validation f1 of 0.666. Saving model.
Epoch 2: train_loss: 0.506 train_precision: 0.797 train_recall: 0.719 train_f1: 0.756 train_acc: 0.768
	 valid_loss: 0.644 valid_precision: 0.728 valid_recall: 0.613 valid_f1: 0.666 valid_acc: 0.692
	 01:20:06.15


...

Training: 100%|██████████████████████████████████████████████████████████████████| 4124/4124 [1:18:09<00:00,  1.14s/it]


Epoch 35: train_loss: 0.074 train_precision: 0.985 train_recall: 0.987 train_f1: 0.986 train_acc: 0.986
	 valid_loss: 2.054 valid_precision: 0.768 valid_recall: 0.651 valid_f1: 0.705 valid_acc: 0.727
	 01:20:07.04


Training: 100%|██████████████████████████████████████████████████████████████████| 4124/4124 [1:18:20<00:00,  1.14s/it]


Epoch 36: train_loss: 0.061 train_precision: 0.988 train_recall: 0.990 train_f1: 0.989 train_acc: 0.989
	 valid_loss: 2.195 valid_precision: 0.804 valid_recall: 0.578 valid_f1: 0.673 valid_acc: 0.719
	 01:20:18.09


Training:   6%|███▉                                                               | 241/4124 [04:29<1:12:00,  1.11s/it]

In [ ]:
import matplotlib.pyplot as plt

# 假设train_losses和val_losses分别存储了训练损失和验证损失
epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses, 'r-', label='Training Loss')
plt.plot(epochs, valid_losses, 'b-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def retrieve_test_data():
    test_param = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/param/test.json")
    test_return = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/return/test.json")
    test_summary = pd.read_json("D:/LTX/code_comment_inconsistency_detection/data/summary/test.json")
    test_df = pd.concat([test_summary,test_param, test_return], axis=0)
    test_df = test_df[:1000]
    return test_df

In [ ]:
def test(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    predictions = []
    gold_labels = []

    with torch.no_grad():
        for batch_idx, (sequence, attention_masks, token_type_ids, labels) in enumerate(test_loader):
            sequence = sequence.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)

            outputs = model(sequence, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            prediction = outputs.logits

            test_loss += loss.item()
            prediction = torch.argmax(prediction, dim=-1)

            predictions.extend(prediction)
            gold_labels.extend(labels)
        
    test_loss = test_loss / len(test_loader)
    test_precision, test_recall, test_f1, test_acc = compute_metrics(predictions, gold_labels)

    print(f"test_loss: {test_loss:.3f} test_precision: {test_precision:.3f} test_recall: {test_recall:.3f} test_f1: {test_f1:.3f} test_acc: {test_acc:.3f}")

In [ ]:
test_df = retrieve_test_data()
train_df_clean = format_data(train_df)
test_data = CocoDataset(train_df_clean)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

test(model, test_loader, device)